In [1]:
import os
import tensorflow as tf
import tensorflow.keras as K
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline              
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg19 import VGG19

2024-01-01 23:49:30.898742: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-01 23:49:30.934198: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-01 23:49:30.934583: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-01 23:49:31.580451: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# defining the image input and our batch size
input_size = 96
batch_size= 16 

In [3]:
vgg = VGG19(include_top=False, weights='imagenet', input_shape=(None,None,3))
vgg_layer = K.Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
for l in vgg_layer.layers: l.trainable=False 

def perceptual_loss(y_true,y_pred):
    y_t=vgg_layer(y_true)
    y_p=vgg_layer(y_pred)
    loss=K.losses.mean_squared_error(y_t,y_p)
    return loss

2024-01-01 23:49:37.846346: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-01 23:49:37.846625: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
def psnr(y_true,y_pred):
    return tf.image.psnr(y_true,y_pred,1.0)
def ssim(y_true,y_pred):
    return tf.image.ssim(y_true,y_pred,1.0)

In [5]:
def image_proc_net(training_mode=True):
    inputs = K.layers.Input((None, None, 3))
    if training_mode:
        x = K.layers.MaxPool2D(pool_size=(2,2))(inputs)
        x = K.layers.GaussianNoise(5)(x)
        x = K.layers.UpSampling2D((2,2))(x)
    else:
        x = K.layers.UpSampling2D((2,2))(inputs)

    model = K.models.Model(inputs, x)

    for l in model.layers: l.trainable=False

    return model

image_proc_train = image_proc_net(training_mode=True)
image_proc_test = image_proc_net(training_mode=False)

In [6]:
print(image_proc_train.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, None, None, 3)    0         
 )                                                               
                                                                 
 gaussian_noise (GaussianNoi  (None, None, None, 3)    0         
 se)                                                             
                                                                 
 up_sampling2d (UpSampling2D  (None, None, None, 3)    0         
 )                                                               
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________

In [7]:
print(image_proc_test.summary())

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 up_sampling2d_1 (UpSampling  (None, None, None, 3)    0         
 2D)                                                             
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
def image_preprocess(image, training_mode=True):
    image = np.expand_dims(image,axis=0)
    if training_mode:
        return tf.squeeze(image_proc_train(image))
    else:
        return tf.squeeze(image_proc_test(image))

In [9]:
def read_image(img_path):
    img = cv2.imread(img_path)
    b,g,r = cv2.split(img)   # cv2 reads BGR instead of canonical RGB
    img = cv2.merge([r,g,b]) # Switching it to RGB

    return img

In [10]:
all_images = []
train_list = []
val_list = []

path = './archive/train_images/'
all_images = [f'./archive/train_images/{f}' for f in os.listdir(path) if f.endswith(".png") or f.endswith(".jpg")]
np.random.shuffle(all_images)  # random shuffling


# keep 65% for train, 20% for testing and 15% for validation
train_len = int(len(all_images)*0.65)

train_list = all_images[:train_len]
val_list = all_images[train_len: train_len + int(train_len * 0.2)]

test_list = []

test_list = all_images[train_len + int(train_len * 0.2):]

print('images: ')
print('train:', len(train_list), 'val:', len(val_list), 'test:', len(test_list))

images: 
train: 3250 val: 650 test: 1100


In [11]:
# defining train, validation and test generator

def train_generator():
    global batch_size
    while True:
        for start in range(0, len(train_list), batch_size):
                    x_batch = []
                    y_batch = []
                    end = min(start + batch_size, len(train_list))
                    ids_train_batch = train_list[start:end]
                    for i,ids in enumerate(ids_train_batch):
                        img_y = read_image(ids)
                        img_x = image_preprocess(img_y, training_mode=True)
                        x_batch.append(np.array(img_x,np.float32)/255.)
                        y_batch.append(np.array(img_y,np.float32)/255.)
                    x_batch = np.array(x_batch)
                    y_batch = np.array(y_batch)
                    yield x_batch,y_batch
                    
def valid_generator():
    global batch_size
    while True:
        for start in range(0, len(val_list), batch_size):
                    x_batch = []
                    y_batch = []
                    end = min(start + batch_size, len(val_list))
                    ids_val_batch = val_list[start:end]
                    for i,ids in enumerate(ids_val_batch):
                        img_y = read_image(ids)
                        img_x = image_preprocess(img_y, training_mode=True)
                        x_batch.append(np.array(img_x,np.float32)/255.)
                        y_batch.append(np.array(img_y,np.float32)/255.)
                    x_batch = np.array(x_batch)
                    y_batch = np.array(y_batch)
                    yield x_batch,y_batch


def test_generator():
    global batch_size
    while True:
        for start in range(0, len(test_list), batch_size):
                    x_batch = []
                    y_batch = []
                    end = min(start + batch_size, len(test_list))
                    ids_test_batch = test_list[start:end]
                    for i,ids in enumerate(ids_test_batch):
                        img_y = read_image(ids)
                        img_x = image_preprocess(img_y, training_mode=False)
                        x_batch.append(np.array(img_x,np.float32)/255.)
                        y_batch.append(np.array(img_y,np.float32)/255.)
                    x_batch = np.array(x_batch)
                    y_batch = np.array(y_batch)
                    yield x_batch,y_batch

In [12]:
# pixel shuffle 
def pixel_shuffle(scale):
    if scale > 1:
        return lambda x: tf.nn.depth_to_space(x, scale)
    else:
        return lambda x:x

In [13]:
def add_down_block(x_inp, filters, kernel_size=(3, 3), padding="same", strides=1,r=False):
    x = K.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x_inp)
    x = K.layers.BatchNormalization()(x)
    x = K.layers.Activation('relu')(x)
    x = K.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    x = K.layers.BatchNormalization()(x)
    if r:
        # if r=True then we import an (1X1) Conv2D after input layer 
        # in order the dimensions of 2 tensors coincide.
        x_inp = K.layers.Conv2D(filters,(1,1), padding=padding, strides=strides)(x_inp)
    x = K.layers.Add()([x,x_inp])
    return x

def add_up_block(x_inp,skip,filters, kernel_size=(3, 3), padding="same", strides=1,upscale_factor=2):
    x = pixel_shuffle(scale=upscale_factor)(x_inp)
    x = K.layers.Concatenate()([x, skip])
    x = K.layers.BatchNormalization()(x)
    x = K.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    x = K.layers.Activation('relu')(x)
    x = K.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    x = K.layers.Activation('relu')(x)
    x = K.layers.Activation('relu')(x)
    return x

def add_bottleneck(x_inp,filters, kernel_size=(3, 3), padding="same", strides=1):
    x = K.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x_inp)
    x = K.layers.Activation('relu')(x)
    return x

In [14]:
def RUNet():
    inputs = K.Input((input_size,input_size, 3))
    
    
    down_1 = K.layers.Conv2D(64,(7,7), padding="same", strides=1)(inputs)
    down_1 = K.layers.BatchNormalization()(down_1)
    down_1 = K.layers.Activation('relu')(down_1)
    
    down_2 = K.layers.MaxPool2D(pool_size=(2,2))(down_1)
    down_2 = add_down_block(down_2,64)
    down_2 = add_down_block(down_2,64)
    down_2 = add_down_block(down_2,64)
    down_2 = add_down_block(down_2,128,r=True)
    
    down_3 = K.layers.MaxPool2D(pool_size=(2, 2),strides=2)(down_2)
    down_3 = add_down_block(down_3,128)
    down_3 = add_down_block(down_3,128)
    down_3 = add_down_block(down_3,128)
    down_3 = add_down_block(down_3,256,r=True)
    
    down_4 = K.layers.MaxPool2D(pool_size=(2, 2))(down_3)
    down_4 = add_down_block(down_4,256)
    down_4 = add_down_block(down_4,256)
    down_4 = add_down_block(down_4,256)
    down_4 = add_down_block(down_4,256)
    down_4 = add_down_block(down_4,256)
    down_4 = add_down_block(down_4,512,r=True) 
    
    down_5 = K.layers.MaxPool2D(pool_size=(2, 2))(down_4)
    down_5 = add_down_block(down_5,512)
    down_5 = add_down_block(down_5,512)
    down_5 = K.layers.BatchNormalization()(down_5)
    down_5 = K.layers.Activation('relu')(down_5)
    
    
    bn_1 = add_bottleneck(down_5, 1024)
    bn_2 = add_bottleneck(bn_1, 512)
    
    up_1 = add_up_block(bn_2,down_5, 512,upscale_factor=1)
    up_2 = add_up_block(up_1,down_4, 384,upscale_factor=2)
    up_3 = add_up_block(up_2,down_3, 256,upscale_factor=2)
    up_4 = add_up_block(up_3,down_2, 96,upscale_factor=2) 
    
    up_5 = pixel_shuffle(scale=2)(up_4)
    up_5 = K.layers.Concatenate()([up_5,down_1])
    up_5 = K.layers.Conv2D(99,(3,3), padding="same", strides=1)(up_5)
    up_5 = K.layers.Activation('relu')(up_5)
    up_5 = K.layers.Conv2D(99,(3,3), padding="same", strides=1)(up_5)
    up_5 = K.layers.Activation('relu')(up_5)
   
    outputs = K.layers.Conv2D(3,(1,1), padding="same")(up_5)
    model = K.models.Model(inputs, outputs)
    return model

model = RUNet()
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 96, 96, 64)   9472        ['input_4[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 96, 96, 64)  256         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 96, 96, 64)   0           ['batch_normalization[0][0]

 activation_5 (Activation)      (None, 24, 24, 128)  0           ['batch_normalization_9[0][0]']  
                                                                                                  
 conv2d_11 (Conv2D)             (None, 24, 24, 128)  147584      ['activation_5[0][0]']           
                                                                                                  
 batch_normalization_10 (BatchN  (None, 24, 24, 128)  512        ['conv2d_11[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_4 (Add)                    (None, 24, 24, 128)  0           ['batch_normalization_10[0][0]', 
                                                                  'max_pooling2d_2[0][0]']        
                                                                                                  
 conv2d_12

 batch_normalization_20 (BatchN  (None, 12, 12, 256)  1024       ['conv2d_22[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 add_9 (Add)                    (None, 12, 12, 256)  0           ['batch_normalization_20[0][0]', 
                                                                  'add_8[0][0]']                  
                                                                                                  
 conv2d_23 (Conv2D)             (None, 12, 12, 256)  590080      ['add_9[0][0]']                  
                                                                                                  
 batch_normalization_21 (BatchN  (None, 12, 12, 256)  1024       ['conv2d_23[0][0]']              
 ormalization)                                                                                    
          

                                                                  'max_pooling2d_4[0][0]']        
                                                                                                  
 conv2d_34 (Conv2D)             (None, 6, 6, 512)    2359808     ['add_14[0][0]']                 
                                                                                                  
 batch_normalization_31 (BatchN  (None, 6, 6, 512)   2048        ['conv2d_34[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_16 (Activation)     (None, 6, 6, 512)    0           ['batch_normalization_31[0][0]'] 
                                                                                                  
 conv2d_35 (Conv2D)             (None, 6, 6, 512)    2359808     ['activation_16[0][0]']          
          

                                                                                                  
 tf.nn.depth_to_space_2 (TFOpLa  (None, 48, 48, 64)  0           ['activation_28[0][0]']          
 mbda)                                                                                            
                                                                                                  
 concatenate_3 (Concatenate)    (None, 48, 48, 192)  0           ['tf.nn.depth_to_space_2[0][0]', 
                                                                  'add_3[0][0]']                  
                                                                                                  
 batch_normalization_37 (BatchN  (None, 48, 48, 192)  768        ['concatenate_3[0][0]']          
 ormalization)                                                                                    
                                                                                                  
 conv2d_44

In [ ]:
opt=K.optimizers.Adam(learning_rate=0.001) # Adam optimizer
model.compile(optimizer=opt,loss=perceptual_loss,metrics=[psnr,ssim,K.losses.mean_squared_error])
history = model.fit_generator(generator=train_generator(),
                              steps_per_epoch=np.ceil(float(len(train_list)) / float(batch_size)),
                              epochs=20,
                              verbose=1,
                              validation_data=valid_generator(),
                              shuffle=True,
                              validation_steps=np.ceil(float(len(val_list)) / float(batch_size)))

# history = model.fit(train_generator, validation_data=valid_generator, batch_size = 16, epochs = 20)

Epoch 1/20


/tmp/ipykernel_17790/1671785620.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_generator(),
2024-01-01 23:50:39.083248: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


  4/204 [..............................] - ETA: 15:56 - loss: 63.9433 - psnr: 5.1910 - ssim: 0.0097 - mean_squared_error: 0.3687